<h1 style="text-align: center;">STACKOVERFLOW DATA CLEANING</h1>

## 0. PACKAGE REQUIREMENT LIST

In [1]:
# Package requirement list

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

/Users/emilialautz/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 1. ORIGINAL DATA: ANALYSIS & CLEANING (DON'T RUN)

In [2]:
# Read the CSV file into a DataFrame
#file_path = '../survey_results_public_22.csv'
#candidates_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
#candidates_df[:10]

In [3]:
#candidates_df.isna().sum()

In [4]:
#candidates_df.dtypes

In [6]:
# Filter only for variables of interest
#candidates_df_selected = candidates_df[["MainBranch", "EdLevel", "YearsCode", "YearsCodePro", "DevType", 
#                                        "LanguageHaveWorkedWith", "ConvertedCompYearly", "Gender"]]

#candidates_df_selected[:10]

In [7]:
#candidates_df_selected.isna().sum()

In [8]:
#candidates_df_selected.dtypes

In [9]:
# Save candidates_df_selected as a CSV file
#candidates_df_selected.to_csv('raw_data/stackoverflow_data_raw.csv', index=False)

## 2. CREATE NEW VARIABLES

- "Degree"
    - From EdLevel
- "Software_Programming" 
    - 1: if the answer to "MainBranch" is: 
        - I am a developer by profession
        - I am not primarily a developer, but I write code sometimes as part of my work/studies - I used to be a developer by profession, but no longer am
        - I code primarily as a hobby
    - 0: otherwise
- "C_Programming" (C or C++)
    - 1: if C or C++ in LanguageHaveWorkedWith
    - 0: otherwise
- "Python_Programming"
    - 1: if Python in LanguageHaveWorkedWith
    - 0: otherwise
- "JavaScript_Programming"
    - 1: if JavaScript in LanguagesHaveWorkedWith
    - 0: otherwise
- "Professional_Software_Experience"
    - From YearsCodePro
- "Management_Skills"
    - 1: if "manager" or "management" in DevType
    - 0: otherwise
- "Engineer"
    - 1: if "Engineer" in DevType
    - 0: otherwise
- "Previous_Pay"
    - From ConvertedCompYearly

In [2]:
# Read the CSV file into a DataFrame
file_path = 'raw_data/stackoverflow_data_raw.csv'
candidates_df = pd.read_csv(file_path)

candidates_df_filtered = candidates_df.dropna(subset=[col for col in candidates_df.columns if col != 'ConvertedCompYearly'])
candidates_df_filtered = candidates_df_filtered.copy()

candidates_df_filtered.isna().sum()

MainBranch                    0
EdLevel                       0
YearsCode                     0
YearsCodePro                  0
DevType                       0
LanguageHaveWorkedWith        0
ConvertedCompYearly       13019
Gender                        0
dtype: int64

In [3]:
len(candidates_df_filtered)

50597

### A. Degree

In [4]:
def check_degree(value):
    """Check if the row contains a degree and return the degree level.
    "No_Degree": No degree
    "Bachelor": Bachelor's degree
    "Master": Master's degree
    "Other": Other degree
    np.nan: Missing value
    """
    
    if isinstance(value, str):
        value = value.lower()
        if "bachelor" in value:
            return "Bachelor"
        elif "master" in value:
            return "Master"       
        elif "professional degree" in value:
            return "Other"
        elif "associate degree" in value:
            return "Other"
        else:
            return "No_Degree"
    return np.nan

# Apply the function to each row to create the 'Degree' column
candidates_df_filtered['Degree'] = candidates_df_filtered['EdLevel'].apply(check_degree)

# One-hot encode the degree variable (No_Degree as baseline)
candidates_df_filtered['Degree'] = pd.Categorical(candidates_df_filtered['Degree'], categories=['No_Degree', 'Bachelor', 'Master', 'Other'], ordered=True)
candidates_df_filtered = pd.get_dummies(candidates_df_filtered, columns=['Degree'], drop_first=True)

### B. Software Programming

In [5]:
def check_software_programming(value):
    """Check if the value contains experience in software programming.
    1: if the answer to the question "MainBranch" is
        - I am a developer by profession
        - I am not primarily a developer, but I write code sometimes as part of my work/studies
        - I used to be a developer by profession, but no longer am
        - I code primarily as a hobby
    0: otherwise (no missing values)
    """

    if value in ["I am a developer by profession", 
                 "I am not primarily a developer, but I write code sometimes as part of my work/studies",
                 "I used to be a developer by profession, but no longer am",
                 "I code primarily as a hobby"]:
        return 1
    return 0

# Apply the function to each row to create the 'Software_Programming' column
candidates_df_filtered['Software_Programming'] = candidates_df_filtered['MainBranch'].apply(check_software_programming)

### C. C_Programming

In [6]:
def check_c_programming(value):
    """Check if the value contains experience in C programming.
    1: if the answer to the question "LanguageHaveWorkedWith" contains "C" or "C++"
    0: otherwise (including missing values)
    """

    if isinstance(value, str):
        value = value.lower()
        if "c;" in value or "c++" in value or "c " in value:
            return 1
    return 0

# Apply the function to each row to create the 'C_Programming' column
candidates_df_filtered['C_Programming'] = candidates_df_filtered['LanguageHaveWorkedWith'].apply(check_c_programming)

### D. Python_Programming

In [7]:
def check_python_programming(value):
    """Check if the value contains experience in Python programming.
    1: if the answer to the question "LanguageHaveWorkedWith" contains "Python"
    0: otherwise (including missing values)
    """

    if isinstance(value, str):
        value = value.lower()
        if "python" in value:
            return 1
    return 0

# Apply the function to each row to create the 'Python_Programming' column
candidates_df_filtered['Python_Programming'] = candidates_df_filtered['LanguageHaveWorkedWith'].apply(check_python_programming)

### E. JavaScript_Programming

In [8]:
def check_javascript_programming(value):
    """Check if the value contains experience in JavaScript programming.
    1: if the answer to the question "LanguageHaveWorkedWith" contains "JavaScript"
    0: otherwise (including missing values)
    """

    if isinstance(value, str):
        value = value.lower()
        if "javascript" in value:
            return 1
    return 0

# Apply the function to each row to create the 'JavaScript_Programming' column
candidates_df_filtered['JavaScript_Programming'] = candidates_df_filtered['LanguageHaveWorkedWith'].apply(check_javascript_programming)

### F. Professional Software Experience

In [9]:
def check_professional_software_experience(value):
    """Extracts the number of years of non-internship professional software development experience.
    """

    if isinstance(value, str):
        if "Less than 1 year" in value:
            return 0
        elif "More than 50 years" in value:
            return 51 
        else:
            return int(value)
    
    return np.nan

# Apply the function to each row to create the 'Professional_Software_Experience' column
candidates_df_filtered['Professional_Software_Experience'] = candidates_df_filtered['YearsCodePro'].apply(check_professional_software_experience)

### G. Management Skills 



In [10]:
def check_management_skills(value):
    """Check if the value contains experience in management skills.
    1: if the answer to the question "DevType" contains "manager"
    0: otherwise (including missing values)
    """

    if isinstance(value, str):
        value = value.lower()
        if "manager" in value:
            return 1
    return 0

# Apply the function to each row to create the 'Management_Skills' column
candidates_df_filtered['Management_Skills'] = candidates_df_filtered['DevType'].apply(check_management_skills)

### H. Engineer

In [11]:
def check_engineer(value):
    """Check if the value contains experience in engineering.
    1: if the answer to the question "DevType" contains "engineer"
    0: otherwise (including missing values)
    """

    if isinstance(value, str):
        value = value.lower()
        if "engineer" in value and "manager" not in value:
            return 1
    return 0

# Apply the function to each row to create the 'Engineer' column
candidates_df_filtered['Engineer'] = candidates_df_filtered['DevType'].apply(check_engineer)

### Previous Pay

In [12]:
def check_previous_pay(value):
    """Extracts the previous yearly compensation from the variable "ConvertedCompYearly" (in USD).
    """

    return pd.to_numeric(value, errors='coerce')
    
# Apply the function to each row to create the 'Previous_Pay' column
candidates_df_filtered['Previous_Pay'] = candidates_df_filtered['ConvertedCompYearly'].apply(check_previous_pay)

### Gender

In [13]:
candidates_df_filtered["Gender"].value_counts()

Man                                                                                   46676
Woman                                                                                  2329
Prefer not to say                                                                       654
Non-binary, genderqueer, or gender non-conforming                                       389
Or, in your own words:                                                                  154
Man;Non-binary, genderqueer, or gender non-conforming                                   138
Man;Or, in your own words:                                                              102
Woman;Non-binary, genderqueer, or gender non-conforming                                  92
Man;Woman;Non-binary, genderqueer, or gender non-conforming                              19
Man;Woman                                                                                10
Or, in your own words:;Non-binary, genderqueer, or gender non-conforming        

In [14]:
def recode_gender(gender):
    """Clean and numerically encode the gender variable.
    1: "Woman"
    2: "Man"
    3: "Prefer not to say" 
    0: otherwise
    """
    if gender == 'Woman':
        return 1
    elif gender == 'Man':
        return 2
    elif gender == 'Prefer not to say':
        return 3
    else:
        return 0

candidates_df_filtered['Gender'] = candidates_df_filtered['Gender'].apply(recode_gender)

In [15]:
candidates_df_filtered["Gender"].value_counts()

2    46676
1     2329
0      938
3      654
Name: Gender, dtype: int64

## 3. IMPUTE MISSING VALUES IN 'Previous_Pay'

- Use a regression to impute missing values in 'Previous_Pay'
- Predictors:
    - 'Gender', 
    - 'Degree_Bachelor', 
    - 'Degree_Master',
    - 'Degree_Other',
    - 'Software_Programming', 
    - 'C_Programming', 
    - 'Python_Programming', 
    - 'JavaScript_Programming',
    - 'Professional_Software_Experience', 
    - 'Management_Skills', 
    - 'Engineer'

In [16]:
candidates_df_imputed_gender = candidates_df_filtered.copy()
candidates_df_imputed_gender[:5]

,MainBranch,EdLevel,YearsCode,YearsCodePro,DevType,LanguageHaveWorkedWith,ConvertedCompYearly,Gender,Degree_Bachelor,Degree_Master,Degree_Other,Software_Programming,C_Programming,Python_Programming,JavaScript_Programming,Professional_Software_Experience,Management_Skills,Engineer,Previous_Pay
2,"I am not primarily a developer, but I write co...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",14,5,Data scientist or machine learning specialist;...,C#;C++;HTML/CSS;JavaScript;Python,40205.0,2,0,1,0,0,1,1,1,5,0,1,40205.0
3,I am a developer by profession,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",20,17,"Developer, full-stack",C#;JavaScript;SQL;TypeScript,215232.0,2,1,0,0,1,0,0,1,17,0,0,215232.0
9,I am a developer by profession,Some college/university study without earning ...,37,30,"Developer, desktop or enterprise applications;...",Delphi;Java;Swift,NaN,1,0,0,0,1,0,0,0,30,0,0,NaN
10,I am a developer by profession,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,2,"Developer, full-stack;Developer, back-end",Bash/Shell;C#;HTML/CSS;JavaScript;PowerShell;SQL,60307.0,2,1,0,0,1,0,0,1,2,0,0,60307.0
11,"I am not primarily a developer, but I write co...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",12,10,Engineering manager,C#;HTML/CSS;JavaScript;PowerShell;Python;Rust;SQL,194400.0,2,1,0,0,0,0,1,1,10,1,0,194400.0


In [17]:
# Filter out categories 0 and 3 in the Gender Variable
candidates_df_imputed_gender = candidates_df_imputed_gender.loc[candidates_df_imputed_gender['Gender'].isin([1, 2])]

In [18]:
len(candidates_df_imputed_gender)

49005

In [19]:
candidates_df_imputed_gender.isna().sum()

MainBranch                              0
EdLevel                                 0
YearsCode                               0
YearsCodePro                            0
DevType                                 0
LanguageHaveWorkedWith                  0
ConvertedCompYearly                 12495
Gender                                  0
Degree_Bachelor                         0
Degree_Master                           0
Degree_Other                            0
Software_Programming                    0
C_Programming                           0
Python_Programming                      0
JavaScript_Programming                  0
Professional_Software_Experience        0
Management_Skills                       0
Engineer                                0
Previous_Pay                        12495
dtype: int64

In [21]:
# Separate the columns to be used for prediction
predictors = ['Gender', 
              'Degree_Bachelor', 
              'Degree_Master',
              'Degree_Other',
              'Software_Programming', 
              'C_Programming', 
              'Python_Programming', 
              'JavaScript_Programming',
              'Professional_Software_Experience', 
              'Management_Skills', 
              'Engineer']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Gender'])
    ],
    remainder='passthrough'
)

# Create the pipeline with the preprocessor and the regressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Function to impute missing values using the regression model
def impute_missing_values(df, target):

    # Train test split
    train_data = df[df[target].notna()]
    predict_data = df[df[target].isna()]

    # Fit the model on the training data
    model.fit(train_data[predictors], train_data[target])

    # Predict the missing values
    df.loc[df[target].isna(), target] = model.predict(predict_data[predictors])

# Impute the missing values in the 'Previous_Pay' column
impute_missing_values(candidates_df_imputed_gender, 'Previous_Pay')

In [22]:
# Get the mean of the 'Previous_Pay' column by gender
candidates_df_imputed_gender.groupby('Gender')['Previous_Pay'].mean()

Gender
1    153607.655279
2    169045.981047
Name: Previous_Pay, dtype: float64

## 4. STORE RESULTS

In [24]:
# Write the cleaned data to a CSV file
candidates_df_imputed_gender.to_csv('cleaned_data/candidates_cleaned.csv', index=False)